In [1]:
import pandas as pd
import numpy as np
# koNLPy에서 제공하는 mecab은 윈도우에서 사용이 불가능 -> eunjeon을 이용함
from eunjeon import Mecab
import re
from math import log
# 형태소 : 최소 의미 단위`

In [2]:
# 형태소 분석, TF-IDF 분석할 문서(리뷰 csv 파일 받아오기)
docs = pd.read_csv('mask_data_final.csv')

In [7]:
import csv

with open('vocab.csv', newline='') as f:
    reader = csv.reader(f)
    data = list(reader)

[['ㄲㄱ', 'ㄲㄲ', 'ㄲㄲㄲ', 'ㄲㄲ굿ㄲㄲ굿ㄲㄲ굿ㄲㄲ굿', 'ㄲㄴ', 'ㄲㄴㅎ어지지만', 'ㄲㅁ', 'ㄲㅁ라는', 'ㄲㅁ랑', 'ㄲㅈ', 'ㄲㅋㅋ', 'ㄲㅌ', 'ㄲㅏ', 'ㄲㅏㄹ', 'ㄲㅏ슬', 'ㄲㅏ지', 'ㄲㅑ', 'ㄲㅓ', 'ㄲㅓ같아여', 'ㄲㅓ라', 'ㄲㅓ보다', 'ㄲㅔ', 'ㄲㅔ여', 'ㄲㅔ요', 'ㄲㅔ요너무좋네여', 'ㄲㅔ요서비스', 'ㄲㅔ용', 'ㄲㅕ도', 'ㄲㅕ봣오요', 'ㄲㅕ요', 'ㄲㅕ주네요', 'ㄲㅣ고', 'ㄲㅣ면', 'ㄲ가', 'ㄲ가카이', 'ㄲ갉끔합니다', 'ㄲ고', 'ㄲ굿굿ㅋㅋ', 'ㄲ굿입니다', 'ㄲ기요ㅡ', 'ㄲ뇨', 'ㄲ다가', 'ㄲ요', 'ㄲ요다른거보다', 'ㄳㄳ', 'ㄴ다', 'ㄴ다고', 'ㄴ데', 'ㄴ디', 'ㄴ지', 'ㄶㄴ', 'ㄶㅇ', 'ㄶ겠지요', 'ㄶ고', 'ㄸㄷ기는', 'ㄸㅎ', 'ㄸㅏㄱ이예요', 'ㄸㅏ가워서', 'ㄸㅐ', 'ㄸㅐ가', 'ㄸㅐ도', 'ㄸㅐ마다', 'ㄸㅐ쓰기', 'ㄸㅓ요', 'ㄸㅗ', 'ㄸ나', 'ㄸ네요', 'ㄸ늑', 'ㄸ로', 'ㄸ습니다', 'ㄸ쓸듯', 'ㄸ아요', 'ㄹ게요', 'ㄹ고', 'ㄹ께', 'ㄹ라', 'ㄹ랑', 'ㄹ래', 'ㄻㄴ', 'ㄻㅇ', 'ㅁ마', 'ㅁ서', 'ㅁ에도', 'ㅂ니', 'ㅂ니다', 'ㅃㄹㄹ범벅', 'ㅃㅃ', 'ㅃㅏ', 'ㅃㅏㄹ라요', 'ㅃㅏ르게', 'ㅃㅏ르게잘받았습니다', 'ㅃㅏ르고', 'ㅃㅏ르고재질', 'ㅃㅏ르고편해요', 'ㅃㅏ르구용ㅋㅋ', 'ㅃㅏ르네요', 'ㅃㅏ른', 'ㅃㅏ른배송', 'ㅃㅏ른배송과', 'ㅃㅐ고', 'ㅃㅓ서', 'ㅃㅓ여', 'ㅃㅓ요', 'ㅃㅓ요대형이', 'ㅃㅓ요많이파세용', 'ㅃㅡ고', 'ㅃㅡ고좋아요', 'ㅃㅡ구', 'ㅃ고', 'ㅃ구', 'ㅃ다고', 'ㅃ룬', 'ㅃ르고', 'ㅃ른배송', 'ㅃ빨리와서좋았어욬ㅋ', 'ㅃ싸게', 'ㅃ어오', 'ㅃ지않아요', 'ㅃ품입니다', 'ㅆㄴ', 'ㅆㄴ는데', 'ㅆㄷ', 'ㅆㄷㄹ껭ㄹㄷ', 'ㅆㄷ고있슥니다ㅎㅎ', 'ㅆㄷ면', 'ㅆㄹㄱ', 'ㅆㄹ게요', 'ㅆㄹ께요', 'ㅆㄹ려고', 'ㅆㅂ', 'ㅆㅂ거', 'ㅆㅂ니다', 

In [5]:
r_1 = docs.iloc[:,[-1]]

In [101]:
N = len(r_1)

def tf(t, d):
    return d.count(t)

def idf(t):
    df = 0
    for doc in docs:
        df += t in doc
    return log(N/(df+1))

def tfidf(t, d):
    return tf(t, d) * idf(t)

result = []
for i in range(N):
    result.append([])
    d = r_1['Mask_review'][i]
    for j in range(len(data[0])):
        t = data[0][j]
        result[-1].append(tfidf(t,d))

In [12]:
tfidf_ = pd.DataFrame(result, columns = data[0])
tfidf_.to_csv("r_1.csv")

In [21]:
a1 = pd.read_csv('r_1.csv')

In [22]:
a1.head(1)

,Unnamed: 0,ㄲㄱ,ㄲㄲ,ㄲㄲㄲ,ㄲㄲ굿ㄲㄲ굿ㄲㄲ굿ㄲㄲ굿,ㄲㄴ,ㄲㄴㅎ어지지만,ㄲㅁ,ㄲㅁ라는,ㄲㅁ랑,...,힘주,힘줘,힘줘서,힘차,힘찬,힙합,힜는데,힜던,힜으면,힝크
0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [24]:
temp_1 = a1.loc[:,['편함', '편하', '편해요', '편합니다', '편안', '착용감', '숨쉬기', '적당']]

In [47]:
temp_1.head()

,편함,편하,편해요,편합니다,편안,착용감,숨쉬기,적당
0,152.458055,830.049408,279.506433,84.698919,194.807514,169.397838,770.760165,135.518271
1,25.409676,398.084920,186.337622,59.289243,76.229027,160.927946,211.747298,118.578487
2,0.000000,76.229027,16.939784,8.469892,25.409676,0.000000,8.469892,16.939784
3,0.000000,287.976325,127.048379,50.819352,59.289243,67.759135,135.518271,42.349460
4,25.409676,381.145136,84.698919,42.349460,76.229027,84.698919,296.446217,59.289243


In [48]:
temp_2 = a1.loc[:,['빠르', '빠른', '빠릅니다', '빠름', '빠른데','빨라서','빨라요','빨랐']]
temp_2.head()

,빠르,빠른,빠릅니다,빠름,빠른데,빨라서,빨라요,빨랐
0,228.687082,143.988163,42.349460,8.469892,0.000000,8.469892,59.289243,50.819352
1,254.096758,67.759135,8.469892,0.000000,0.000000,8.469892,50.819352,16.939784
2,42.349460,16.939784,8.469892,0.000000,0.000000,0.000000,16.939784,0.000000
3,118.578487,42.349460,16.939784,8.469892,0.000000,0.000000,25.409676,16.939784
4,245.626866,237.156974,16.939784,8.469892,8.469892,25.409676,25.409676,8.469892


In [51]:
temp_3 = a1.loc[:,['중독', '항상', '추천', '성공', '행복', '만족', '정착']]
temp_3.head()

,중독,항상,추천,성공,행복,만족,정착
0,0.0,152.458055,135.518271,16.939784,16.939784,677.591353,135.518271
1,0.0,76.229027,42.349460,0.000000,0.000000,271.036541,0.000000
2,0.0,50.819352,0.000000,0.000000,0.000000,84.698919,16.939784
3,0.0,33.879568,0.000000,0.000000,0.000000,271.036541,16.939784
4,0.0,59.289243,50.819352,0.000000,0.000000,245.626866,0.000000


In [53]:
temp_4 = a1.loc[:,['저렴', '성비', '고퀄', '퀄리티', '훌륭', '최고', '최고봉']]
temp_4.head()

,저렴,성비,고퀄,퀄리티,훌륭,최고,최고봉
0,287.976325,118.578487,25.409676,42.349460,8.469892,110.108595,8.469892
1,67.759135,67.759135,0.000000,0.000000,0.000000,8.469892,0.000000
2,16.939784,0.000000,0.000000,0.000000,0.000000,16.939784,0.000000
3,76.229027,25.409676,0.000000,16.939784,8.469892,8.469892,0.000000
4,355.735461,84.698919,0.000000,0.000000,8.469892,42.349460,0.000000


In [56]:
temp_6 = a4.loc[:,['부드러운', '부드러움', '부드럽', '부드러워서', '부드러웠']]
temp_6.head()

,부드러운,부드러움,부드럽,부드러워서,부드러웠
0,76.229027,16.939784,135.518271,33.879568,8.469892
1,0.000000,0.000000,8.469892,0.000000,0.000000
2,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.000000,0.000000,42.349460,8.469892,0.000000
4,0.000000,0.000000,76.229027,8.469892,0.000000


In [90]:
review_keyword = pd.concat([temp_1.sum(axis=1),temp_2.sum(axis=1),temp_3.sum(axis=1),temp_4.sum(axis=1),temp_6.sum(axis=1)], axis=1, keys=['편안함','빠른배송','재구매','가성비','부드러움'])
review_keyword

,편안함,빠른배송,재구매,가성비,부드러움
0,2617.196603,542.073083,1134.965517,601.362326,271.036541
1,1236.604220,406.554812,389.615028,143.988163,8.469892
2,152.458055,84.698919,152.458055,33.879568,0.000000
3,770.760165,228.687082,321.855893,135.518271,50.819352
4,1050.266598,575.952650,355.735461,491.253731,84.698919
...,...,...,...,...,...
4764,110.108595,143.988163,118.578487,143.988163,0.000000
4765,0.000000,0.000000,16.939784,0.000000,0.000000
4766,25.409676,0.000000,0.000000,0.000000,0.000000
4767,0.000000,16.939784,0.000000,0.000000,0.000000


In [98]:
review_keyword.to_csv("review_keyword.csv",index = None)